# Name Gender Identifier

## 1. Building a feature extractor

An idea is to use the last letter of the name to predict the gender. For instance, names ending in *a*, *e* and *i* are likely to be female, while names ending in *k*, *o*, *r*, *s* and *t* are likely to be male.

In [4]:
# Feature extractor
def gender_features(word):
    return {'last_letter': word[-1]}

gender_features('John')

{'last_letter': 'n'}

The returned dictionary is known as a **feature set**.

## 2. Exploring the `names` corpus

In [5]:
from nltk.corpus import names

names.readme().replace('\n', ' ')

'Names Corpus, Version 1.3 (1994-03-29) Copyright (C) 1991 Mark Kantrowitz Additions by Bill Ross  This corpus contains 5001 female names and 2943 male names, sorted alphabetically, one per line.  You may use the lists of names for any purpose, so long as credit is given in any published work. You may also redistribute the list if you provide the recipients with a copy of this README file. The lists are not in the public domain (I retain the copyright on the lists) but are freely redistributable.  If you have any additions to the lists of names, I would appreciate receiving them.  Mark Kantrowitz <mkant+@cs.cmu.edu> http://www-2.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/'

In [6]:
names.fileids()

['female.txt', 'male.txt']

In [7]:
names.words('female.txt')[:5]

['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi']

## 3. Building the classifier

We need to prepare a list of examples and corresponding class labels.

In [8]:
labeled_names = ([(name, 'female') for name in names.words('female.txt')] + [(name, 'male') for name in names.words('male.txt')])
labeled_names[:5]

[('Abagael', 'female'),
 ('Abagail', 'female'),
 ('Abbe', 'female'),
 ('Abbey', 'female'),
 ('Abbi', 'female')]

In [9]:
import random
random.shuffle(labeled_names) # We shuffle the data so that we can split it by index into training and test data.
labeled_names[:5]

[('Haskell', 'male'),
 ('Abel', 'male'),
 ('Odella', 'female'),
 ('Theodore', 'male'),
 ('Lawson', 'male')]

In [10]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
featuresets[:5]

[({'last_letter': 'l'}, 'male'),
 ({'last_letter': 'l'}, 'male'),
 ({'last_letter': 'a'}, 'female'),
 ({'last_letter': 'e'}, 'male'),
 ({'last_letter': 'n'}, 'male')]

In [11]:
len(featuresets)

7944

In [12]:
from nltk import NaiveBayesClassifier

# We split the data into a training (80%) and test (20%) set:
TRAIN_SET_SIZE = round(len(featuresets) * .8)
train_set, test_set = featuresets[:TRAIN_SET_SIZE], featuresets[TRAIN_SET_SIZE:]

# We also get the names in the test set, to be used later:
test_names = labeled_names[TRAIN_SET_SIZE:]

classifier = NaiveBayesClassifier.train(train_set)

# When working with large corpora, constructing a single list that contains the features of every instance can use up a large amount of memory. In these cases, use the function nltk.classify.apply_features, which returns an object that acts like a list but does not store all the feature sets in memory: 
# from nltk.classify import apply_features
# train_names, test_names = labeled_names[:round(len(featuresets) * .8)], labeled_names[round(len(featuresets) * .8):]
# train_set = apply_features(gender_features, labeled_names[500:])
# test_set = apply_features(gender_features, labeled_names[:500])

In [13]:
classifier.show_most_informative_features(10) # Prints likelihood ratios for most informative features

Most Informative Features
             last_letter = 'k'              male : female =     35.4 : 1.0
             last_letter = 'a'            female : male   =     32.5 : 1.0
             last_letter = 'f'              male : female =     23.1 : 1.0
             last_letter = 'p'              male : female =     11.8 : 1.0
             last_letter = 'd'              male : female =     10.0 : 1.0
             last_letter = 'o'              male : female =      9.5 : 1.0
             last_letter = 'v'              male : female =      8.4 : 1.0
             last_letter = 'm'              male : female =      8.3 : 1.0
             last_letter = 'z'              male : female =      6.4 : 1.0
             last_letter = 'r'              male : female =      6.4 : 1.0


## 4. Testing the classifier

In [14]:
classifier.labels()

['male', 'female']

In [15]:
from nltk.classify import accuracy

round(accuracy(classifier, test_set), 2)

0.76

In [16]:
classifier.classify(gender_features('Aphrodite'))

'female'

In [17]:
classifier.classify(gender_features('Zeus'))

'male'

## 5. Building a classifier with more features

In [20]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

gender_features2('John')

{'first_letter': 'j',
 'last_letter': 'n',
 'count(a)': 0,
 'has(a)': False,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 1,
 'has(h)': True,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 1,
 'has(j)': True,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [21]:
featuresets2 = [(gender_features2(n), gender) for (n, gender) in labeled_names]
featuresets2[0]

({'first_letter': 'h',
  'last_letter': 'l',
  'count(a)': 1,
  'has(a)': True,
  'count(b)': 0,
  'has(b)': False,
  'count(c)': 0,
  'has(c)': False,
  'count(d)': 0,
  'has(d)': False,
  'count(e)': 1,
  'has(e)': True,
  'count(f)': 0,
  'has(f)': False,
  'count(g)': 0,
  'has(g)': False,
  'count(h)': 1,
  'has(h)': True,
  'count(i)': 0,
  'has(i)': False,
  'count(j)': 0,
  'has(j)': False,
  'count(k)': 1,
  'has(k)': True,
  'count(l)': 2,
  'has(l)': True,
  'count(m)': 0,
  'has(m)': False,
  'count(n)': 0,
  'has(n)': False,
  'count(o)': 0,
  'has(o)': False,
  'count(p)': 0,
  'has(p)': False,
  'count(q)': 0,
  'has(q)': False,
  'count(r)': 0,
  'has(r)': False,
  'count(s)': 1,
  'has(s)': True,
  'count(t)': 0,
  'has(t)': False,
  'count(u)': 0,
  'has(u)': False,
  'count(v)': 0,
  'has(v)': False,
  'count(w)': 0,
  'has(w)': False,
  'count(x)': 0,
  'has(x)': False,
  'count(y)': 0,
  'has(y)': False,
  'count(z)': 0,
  'has(z)': False},
 'male')

In [22]:
train_set2, test_set2 = featuresets2[:TRAIN_SET_SIZE], featuresets2[TRAIN_SET_SIZE:]
classifier2 = NaiveBayesClassifier.train(train_set2)
round(accuracy(classifier2, test_set2), 2)

0.78

We would have expected that having too many specific features on a small dataset would lead to overfitting, but it seems the classifier was good at avoiding that since its performance is slightly better.

In [23]:
classifier2.show_most_informative_features(15)

Most Informative Features
             last_letter = 'k'              male : female =     35.4 : 1.0
             last_letter = 'a'            female : male   =     32.5 : 1.0
             last_letter = 'f'              male : female =     23.1 : 1.0
             last_letter = 'p'              male : female =     11.8 : 1.0
             last_letter = 'd'              male : female =     10.0 : 1.0
             last_letter = 'o'              male : female =      9.5 : 1.0
             last_letter = 'v'              male : female =      8.4 : 1.0
             last_letter = 'm'              male : female =      8.3 : 1.0
                count(v) = 2              female : male   =      8.1 : 1.0
             last_letter = 'z'              male : female =      6.4 : 1.0
             last_letter = 'r'              male : female =      6.4 : 1.0
             last_letter = 'w'              male : female =      5.4 : 1.0
                count(a) = 3              female : male   =      5.2 : 1.0

Indeed, it seems the classifier is mainly using the last letter, along with some other features that happen to improve the accuracy.

## 6. Comparing the two classifiers using `nltk.metrics`

Before we start, here's a useful function for comparing strings:

In [24]:
from nltk.metrics import edit_distance

edit_distance("John", "Joan")

1

The NLTK metrics module provides functions for calculating metrics beyond mere accuracy. But in order to do so, we need to build 2 sets for each classification label: a reference set of correct values, and a test set of observed values.

In [25]:
import collections

# Classifier 1
refsets = collections.defaultdict(set) # For what this is: https://stackoverflow.com/questions/5900578/how-does-collections-defaultdict-work
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    
# Classifier 2
refsets2 = collections.defaultdict(set)
testsets2 = collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set2):
    refsets2[label].add(i)
    observed = classifier2.classify(feats)
    testsets2[observed].add(i)

In [26]:
refsets

defaultdict(set,
            {'female': {0,
              1,
              3,
              4,
              6,
              8,
              9,
              10,
              13,
              14,
              15,
              18,
              22,
              24,
              25,
              26,
              27,
              29,
              31,
              33,
              34,
              35,
              36,
              38,
              39,
              40,
              44,
              45,
              47,
              49,
              50,
              51,
              52,
              53,
              54,
              55,
              56,
              57,
              58,
              59,
              60,
              61,
              62,
              63,
              64,
              65,
              66,
              67,
              68,
              69,
              72,
              73,
              74,
              75,
        

In [27]:
testsets

defaultdict(set,
            {'female': {0,
              1,
              2,
              3,
              4,
              5,
              6,
              7,
              8,
              9,
              10,
              11,
              13,
              14,
              15,
              16,
              17,
              18,
              23,
              25,
              26,
              27,
              28,
              29,
              33,
              34,
              35,
              36,
              37,
              38,
              39,
              40,
              43,
              44,
              45,
              47,
              49,
              50,
              51,
              52,
              55,
              56,
              57,
              58,
              59,
              60,
              61,
              62,
              63,
              65,
              66,
              67,
              68,
              70,
           

In [28]:
from nltk.metrics.scores import (precision, recall, f_measure)

# We can proceed to print the metrics for each classifier. However, we cannot get the accuracy in this manner because nltk.metrics.scores.accuracy(reference, test) works by comparing test[i] == reference[i] and our reference and test are not formatted in a way that allows for this. It's the same for the confusion matrix.
args = (
    round(precision(refsets['female'], testsets['female']), 2),
    round(precision(refsets['male'], testsets['male']), 2),
    round(recall(refsets['female'], testsets['female']), 2),
    round(recall(refsets['male'], testsets['male']), 2),
    round(f_measure(refsets['female'], testsets['female']), 2),
    round(f_measure(refsets['male'], testsets['male']), 2)
)

args2 = (
    round(precision(refsets2['female'], testsets2['female']), 2),
    round(precision(refsets2['male'], testsets2['male']), 2),
    round(recall(refsets2['female'], testsets2['female']), 2),
    round(recall(refsets2['male'], testsets2['male']), 2),
    round(f_measure(refsets2['female'], testsets2['female']), 2),
    round(f_measure(refsets2['male'], testsets2['male']), 2)
)

print('''
CLASSIFIER 1
------------ 
Female precision: {0}
Male precision: {1}
Female recall: {2}
Male recall: {3}
Female F1 score: {4}
Male F1 score: {5}

CLASSIFIER 2
------------ 
Female precision: {6}
Male precision: {7}
Female recall: {8}
Male recall: {9}
Female F1 score: {10}
Male F1 score: {11}
'''.format(*args, *args2))


CLASSIFIER 1
------------ 
Female precision: 0.8
Male precision: 0.68
Female recall: 0.82
Male recall: 0.65
Female F1 score: 0.81
Male F1 score: 0.66

CLASSIFIER 2
------------ 
Female precision: 0.81
Male precision: 0.71
Female recall: 0.85
Male recall: 0.66
Female F1 score: 0.83
Male F1 score: 0.68



## 7. Error analysis

In [29]:
errors = []
for name, tag in test_names:
    guess = classifier2.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))

errors[:5]

[('male', 'female', 'Barclay'),
 ('male', 'female', 'Rodolph'),
 ('male', 'female', 'Douggie'),
 ('male', 'female', 'Kerry'),
 ('male', 'female', 'Way')]

In [30]:
for tag, guess, name in sorted(errors):
    print('Correct = {:8} guess = {:8} name = {}'.format(tag, guess, name)) # :8 creates spaces between columns.

Correct = female   guess = male     name = Abagael
Correct = female   guess = male     name = Aidan
Correct = female   guess = male     name = Alleen
Correct = female   guess = male     name = Alyss
Correct = female   guess = male     name = Anett
Correct = female   guess = male     name = Angil
Correct = female   guess = male     name = Annabal
Correct = female   guess = male     name = Ardeen
Correct = female   guess = male     name = Ardelis
Correct = female   guess = male     name = Arden
Correct = female   guess = male     name = Ardis
Correct = female   guess = male     name = Ardys
Correct = female   guess = male     name = Arleen
Correct = female   guess = male     name = Astrid
Correct = female   guess = male     name = Avis
Correct = female   guess = male     name = Bab
Correct = female   guess = male     name = Beatrix
Correct = female   guess = male     name = Beatriz
Correct = female   guess = male     name = Beilul
Correct = female   guess = male     name = Bel
Correct = 

Looking through this list of errors, it seems that some suffixes that are more than one letter long can be indicative of name genders. For example, names ending in *yn* appear to be predominantly female, despite the fact that names ending in *n* tend to be male; and names ending in *ch* are usually male, even though names that end in *h* tend to be female.

## 8. Building a classifier with even more features

In [31]:
def gender_features3(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["suffix1"] = name[-1].lower()
    features["suffix2"] = name[-2:].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

gender_features3('John')

{'first_letter': 'j',
 'suffix1': 'n',
 'suffix2': 'hn',
 'count(a)': 0,
 'has(a)': False,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 1,
 'has(h)': True,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 1,
 'has(j)': True,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [32]:
featuresets3 = [(gender_features3(n), gender) for (n, gender) in labeled_names]
featuresets3[0]

({'first_letter': 'h',
  'suffix1': 'l',
  'suffix2': 'll',
  'count(a)': 1,
  'has(a)': True,
  'count(b)': 0,
  'has(b)': False,
  'count(c)': 0,
  'has(c)': False,
  'count(d)': 0,
  'has(d)': False,
  'count(e)': 1,
  'has(e)': True,
  'count(f)': 0,
  'has(f)': False,
  'count(g)': 0,
  'has(g)': False,
  'count(h)': 1,
  'has(h)': True,
  'count(i)': 0,
  'has(i)': False,
  'count(j)': 0,
  'has(j)': False,
  'count(k)': 1,
  'has(k)': True,
  'count(l)': 2,
  'has(l)': True,
  'count(m)': 0,
  'has(m)': False,
  'count(n)': 0,
  'has(n)': False,
  'count(o)': 0,
  'has(o)': False,
  'count(p)': 0,
  'has(p)': False,
  'count(q)': 0,
  'has(q)': False,
  'count(r)': 0,
  'has(r)': False,
  'count(s)': 1,
  'has(s)': True,
  'count(t)': 0,
  'has(t)': False,
  'count(u)': 0,
  'has(u)': False,
  'count(v)': 0,
  'has(v)': False,
  'count(w)': 0,
  'has(w)': False,
  'count(x)': 0,
  'has(x)': False,
  'count(y)': 0,
  'has(y)': False,
  'count(z)': 0,
  'has(z)': False},
 'male')

In [33]:
train_set3, test_set3 = featuresets3[:TRAIN_SET_SIZE], featuresets3[TRAIN_SET_SIZE:]
classifier3 = NaiveBayesClassifier.train(train_set3)
round(accuracy(classifier3, test_set3), 2)

0.8

In [34]:
classifier3.show_most_informative_features(15)

Most Informative Features
                 suffix2 = 'na'           female : male   =    145.2 : 1.0
                 suffix2 = 'la'           female : male   =     65.1 : 1.0
                 suffix2 = 'us'             male : female =     60.1 : 1.0
                 suffix2 = 'ia'           female : male   =     43.6 : 1.0
                 suffix1 = 'k'              male : female =     35.4 : 1.0
                 suffix1 = 'a'            female : male   =     32.5 : 1.0
                 suffix2 = 'ra'           female : male   =     29.9 : 1.0
                 suffix2 = 'sa'           female : male   =     29.2 : 1.0
                 suffix2 = 'rd'             male : female =     28.1 : 1.0
                 suffix1 = 'f'              male : female =     23.1 : 1.0
                 suffix2 = 'ta'           female : male   =     21.7 : 1.0
                 suffix2 = 'rt'             male : female =     21.0 : 1.0
                 suffix2 = 'ld'             male : female =     20.2 : 1.0

## 9. Trying to use a maximum entropy classifier

The principle of **maximum entropy** states that the probability distribution which best represents the current state of knowledge is the one with largest entropy.

The principle of maximum entropy is invoked when we have some piece(s) of information about a probability distribution, but not enough to characterize it completely—likely because we do not have the means or resources to do so. As an example, if all we know about a distribution is its average, we can imagine infinite shapes that yield a particular average. The principle of maximum entropy says that we should humbly choose the distribution that maximizes the amount of unpredictability contained in the distribution.

Taking the idea to the extreme, it wouldn’t be scientific to choose a distribution that simply yields the average value 100% of the time.

From all the models that fit our training data, the Maximum Entropy classifier selects the one which has the largest entropy. Due to the minimum assumptions that the Maximum Entropy classifier makes, it is usually used when we don’t know anything about the prior distributions and when it is unsafe to make any assumptions. Also, the maximum entropy classifier is used when we can’t assume the conditional independence of the features.

In [35]:
from nltk import MaxentClassifier

me_classifier = MaxentClassifier.train(train_set3, max_iter=25) # max_iter has default value 100. In this example, the performance in terms of accuracy on the test set starts significantly improving beyond the previous model's at around 25 iterations.

  ==> Training (25 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.372
             2          -0.60306        0.628
             3          -0.58135        0.628
             4          -0.56142        0.636
             5          -0.54321        0.669
             6          -0.52658        0.705
             7          -0.51142        0.731
             8          -0.49760        0.752
             9          -0.48498        0.768
            10          -0.47346        0.779
            11          -0.46291        0.786
            12          -0.45325        0.790
            13          -0.44437        0.797
            14          -0.43620        0.801
            15          -0.42866        0.805
            16          -0.42170        0.805
            17          -0.41525        0.805
            18          -0.40926        0.808
            19          -0.40370        0.809
  

In [36]:
round(accuracy(me_classifier, test_set3), 2) # The accuracies above were on the training set so this is what matters.

0.8

In [37]:
me_classifier.show_most_informative_features(10)

  -2.319 suffix2=='na' and label is 'male'
  -1.902 suffix2=='la' and label is 'male'
  -1.755 suffix2=='us' and label is 'female'
  -1.591 suffix2=='ia' and label is 'male'
  -1.484 suffix2=='ra' and label is 'male'
  -1.477 suffix2=='sa' and label is 'male'
  -1.450 suffix1=='k' and label is 'female'
  -1.396 suffix1=='a' and label is 'male'
  -1.219 suffix1=='f' and label is 'female'
  -1.215 suffix2=='ta' and label is 'male'


## 10. More classifiers

Scikit-learn (sklearn) is a popular library which features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN.

NLTK provides an API to quickly use sklearn classifiers in `nltk.classify.scikitlearn`. The other option is to import and use sklearn directly.

For an example of integrating sklearn with NLTK, you can check out [this](https://www.kaggle.com/alvations/basic-nlp-with-nltk) notebook on Kaggle. Kaggle is a great website for NLP and machine learning in general, creating an account is highly recommended.

In [38]:
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /Users/m/nltk_data...
[nltk_data]   Package names is already up-to-date!


True